In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## TOIs

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [3]:
df = get_tois()
df.tail()

Loaded:  ../data/TOIs.csv


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
5299,428867906,6060.01,NaN,3,4,3,3,1,4,4,...,0.21,0.086,0.070,1.678,0.294829,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,low SNR; two stars in pixel
5300,239823606,6061.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.000,0.127632,"19,43,44,45,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,two stars in pixel; low SNR
5301,122629761,6062.01,NaN,2,4,2,2,1,4,4,...,0.05,NaN,NaN,1.084,0.146070,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,variable host; low SNR
5302,67031480,6063.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,NaN,NaN,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:44,likely EB; a/Rs~2; host is member of M38 cluster
5303,428767018,6064.01,NaN,3,4,3,3,1,4,4,...,0.14,0.133,0.012,1.464,0.245446,"19,59",2023-02-08,2023-02-08,2023-02-10 12:02:43,NaN


In [4]:
tics = df['TIC ID'].unique()
len(tics)

5096

In [5]:
len(df['TOI'].unique())

5304

In [6]:
nsys = df['TIC ID'].value_counts()
nsys

367630162    5
425997655    5
150151262    4
142276270    4
260647166    4
            ..
59842822     1
124491883    1
143282930    1
147430118    1
428767018    1
Name: TIC ID, Length: 5096, dtype: int64

In [7]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

163


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [8]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

830     6.258792
829    12.518728
832    18.804756
831    26.317954
Name: Period (days), dtype: float64

In [9]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [10]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [11]:
resonances_1st_order, deltas = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order, deltas

(['3:2 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
  '2:1 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
  '3:2 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
  '3:2 | P=(39.54,26.32) n=(5,6) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.0016314412161837488])

In [12]:
resonances_2nd_order, deltas = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order, deltas

(['6:4 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
  '4:2 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
  '6:4 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
  '7:5 | P=(26.32,18.80) n=(4,5) (delta=0.01%)',
  '6:4 | P=(39.54,26.32) n=(5,6) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.00010182804131475542,
  0.0016314412161837488])

## All TOIs

In [79]:
ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
for tic in tics_multis:
    dd = df[df['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    toi = str(dd.iloc[0].TOI).split('.')[0] 
    
    resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TOI {toi} | TIC {tic} (nplanets={Nplanets}):<br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
        ttv_1st_order.append(tic)
        deltas1.append(min(delta1))
        if len(resonances_1st_order)>=2:
            ttv_chain.append(tic)         
        show = True
    if len(resonances_2nd_order)>=1:
        msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
        ttv_2nd_order.append(tic)
        deltas2.append(min(delta2))
        show = True
    if show:
        print(msg)
        texts.append(msg)

TOI 2104 | TIC 367630162 (nplanets=5):<br>
Second Order: 5:3 | P=(3.27,1.98) n=(1,2) (delta=0.85%)

TOI 174 | TIC 425997655 (nplanets=5):<br>
First Order: 2:1 | P=(7.91,3.98) n=(1,2) (delta=0.57%)<br>
Second Order: 4:2 | P=(7.91,3.98) n=(1,2) (delta=0.57%)

TOI 1136 | TIC 142276270 (nplanets=4):<br>
First Order: 2:1 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 3:2 | P=(18.80,12.52) n=(2,3) (delta=0.14%)<br>
Second Order: 4:2 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 6:4 | P=(18.80,12.52) n=(2,3) (delta=0.14%) 7:5 | P=(26.32,18.80) n=(3,4) (delta=0.03%)

TOI 700 | TIC 150428135 (nplanets=4):<br>
First Order: 4:3 | P=(37.42,27.81) n=(3,4) (delta=0.93%)<br>
Second Order: 8:6 | P=(37.42,27.81) n=(3,4) (delta=0.93%)

TOI 282 | TIC 29781292 (nplanets=4):<br>
First Order: 3:2 | P=(84.26,56.01) n=(3,4) (delta=0.30%)<br>
Second Order: 6:4 | P=(84.26,56.01) n=(3,4) (delta=0.30%)

TOI 1339 | TIC 269701147 (nplanets=3):<br>
First Order: 4:3 | P=(38.35,28.58) n=(2,3) (delta=0.64%)<br>
Second Order: 8:6 | P=

In [23]:
len(ttv_1st_order), len(ttv_2nd_order)

(18, 22)

In [24]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
9,441798995.0,0.000030,True
1,142276270.0,0.000091,True
14,143022742.0,0.002051,True
3,29781292.0,0.002967,True
15,153065527.0,0.004089,True
8,259377017.0,0.005161,True
0,425997655.0,0.005740,True
13,374180079.0,0.005803,True
16,55652896.0,0.006421,True
4,269701147.0,0.006443,True


In [25]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
12,441798995.0,0.000030,True
2,142276270.0,0.000091,True
10,178155732.0,0.000263,True
17,143022742.0,0.002051,True
4,29781292.0,0.002967,True
18,153065527.0,0.004089,True
9,259377017.0,0.005161,True
1,425997655.0,0.005740,True
16,374180079.0,0.005803,True
20,55652896.0,0.006421,True


In [39]:
res = pd.merge(s1, s2, how='outer')
res = res.replace(np.nan, False)
res['tic'] = res.tic.map(int)
res = res.sort_values(by='delta_min')
res.head()

,tic,delta_min,1st order,2nd order
9,441798995,0.000030,True,True
1,142276270,0.000091,True,True
19,178155732,0.000263,False,True
14,143022742,0.002051,True,True
3,29781292,0.002967,True,True


In [83]:
len(texts), len(res)

(22, 22)

In [81]:
idx = res.index.values
for i in idx:
    print(texts[i])

TOI 270 | TIC 259377017 (nplanets=3):<br>
First Order: 2:1 | P=(11.38,5.66) n=(2,3) (delta=0.52%)<br>
Second Order: 4:2 | P=(11.38,5.66) n=(2,3) (delta=0.52%)

TOI 174 | TIC 425997655 (nplanets=5):<br>
First Order: 2:1 | P=(7.91,3.98) n=(1,2) (delta=0.57%)<br>
Second Order: 4:2 | P=(7.91,3.98) n=(1,2) (delta=0.57%)

TOI 6054 | TIC 392681545 (nplanets=2):<br>
Second Order: 5:3 | P=(12.58,7.49) n=(1,2) (delta=0.74%)

TOI 4495 | TIC 120826158 (nplanets=2):<br>
First Order: 2:1 | P=(5.18,2.57) n=(1,2) (delta=0.86%)<br>
Second Order: 4:2 | P=(5.18,2.57) n=(1,2) (delta=0.86%)

TOI 700 | TIC 150428135 (nplanets=4):<br>
First Order: 4:3 | P=(37.42,27.81) n=(3,4) (delta=0.93%)<br>
Second Order: 8:6 | P=(37.42,27.81) n=(3,4) (delta=0.93%)

TOI 1749 | TIC 233602827 (nplanets=2):<br>
First Order: 2:1 | P=(9.04,4.49) n=(1,2) (delta=0.74%)<br>
Second Order: 4:2 | P=(9.04,4.49) n=(1,2) (delta=0.74%)

TOI 1730 | TIC 318022259 (nplanets=3):<br>
First Order: 2:1 | P=(12.57,6.23) n=(2,3) (delta=0.91%)<br

In [35]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [33]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [ ]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2